In [47]:
import pandas as pd
import numpy as np
import time as time
import requests

import yfinance as yf
# logging
import logging
from datetime import datetime
from datetime import timedelta

In [13]:
# Read the CSV file
sick_df = pd.read_csv('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/all_delistings_since_2020.csv')

# Clean up CIK numbers
sick_df['cik'] = sick_df['cik'].astype(str).str.zfill(10)

# Convert date string to datetime
sick_df['date'] = pd.to_datetime(sick_df['date'])

# Add columns for SEC data
sick_df['ticker'] = ''
sick_df['sic'] = ''
sick_df['sic_description'] = ''
sick_df['is_biotech'] = False
sick_df['biotech_identification_method'] = ''

In [14]:
print(len(sick_df))
display(sick_df.head())

574


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,,,,False,
1,2024-10-21,TELLURIAN INC. /DE/,0000061398,25,,,,False,
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,,,,False,
3,2024-10-11,Investment Managers Series Trust II,0001587982,25,,,,False,
4,2024-10-07,ORANGE,0001038143,25,,,,False,


In [15]:
# Define biotech-related keywords and patterns (expanded)
biotech_keywords = [
    'therapeutics', 'pharma', 'biotech', 'bio', 'health sciences',
    'medical', 'genetics', 'genomics', 'life sciences', 'biologics',
    'biosciences', 'oncology', 'diagnostic', 'biomedical',
    'laboratory', 'clinical', 'research', 'drug', 'therapeutic',
    'cell therapy', 'gene therapy', 'molecular', 'pharmaceutical'
]

In [16]:
# Define comprehensive biotech and related SIC codes with descriptions
biotech_sic_dict = {
    '2833': 'Medicinal Chemicals and Botanical Products',
    '2834': 'Pharmaceutical Preparations',
    '2835': 'In Vitro and In Vivo Diagnostic Substances',
    '2836': 'Biological Products, Except Diagnostic Substances',
    '3826': 'Laboratory Analytical Instruments',
    '3841': 'Surgical and Medical Instruments',
    '3842': 'Orthopedic, Prosthetic, and Surgical Supplies',
    '3843': 'Dental Equipment and Supplies',
    '3844': 'X-Ray Apparatus and Tubes',
    '3845': 'Electromedical and Electrotherapeutic Apparatus',
    '3851': 'Ophthalmic Goods',
    '8071': 'Medical Laboratories',
    '8731': 'Commercial Physical and Biological Research',
    '8732': 'Commercial Nonphysical Research',
    '8733': 'Noncommercial Research Organizations',
    '8734': 'Testing Laboratories',
    '8999': 'Services, Not Elsewhere Classified (including Medical Research)',
    '6324': 'Hospital and Medical Service Plans',
    '5122': 'Drugs, Proprietaries, and Druggists Sundries'
}

biotech_sic_codes = list(biotech_sic_dict.keys())

In [29]:
# Set up SEC API request headers
headers = {
    'User-Agent': 'sanspeursystems matt@sanspeursystems.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'data.sec.gov'
}

# Process each company
for idx, row in sick_df.iterrows():
    print(f"Processing {row['company']} ({idx + 1}/{len(sick_df)})")
    
    # Check company name for biotech keywords
    company_name_lower = row['company'].lower()
    matching_keywords = [kw for kw in biotech_keywords if kw in company_name_lower]
    if matching_keywords:
        sick_df.at[idx, 'is_biotech'] = True
        sick_df.at[idx, 'biotech_identification_method'] = f"name_match:{','.join(matching_keywords)}"
    
    # Get SEC data
    cik = row['cik']
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            
            # Store basic info
            sick_df.at[idx, 'ticker'] = data.get('tickers', [''])[0]
            sic_code = str(data.get('sic', '')).strip()
            sick_df.at[idx, 'sic'] = sic_code
            sick_df.at[idx, 'sic_description'] = data.get('sicDescription', '') or biotech_sic_dict.get(sic_code, '')
            
            # Check if SIC code matches biotech
            if sic_code in biotech_sic_codes:
                sick_df.at[idx, 'is_biotech'] = True
                current_method = sick_df.at[idx, 'biotech_identification_method']
                if current_method:
                    sick_df.at[idx, 'biotech_identification_method'] = f"{current_method};sic_{sic_code}"
                else:
                    sick_df.at[idx, 'biotech_identification_method'] = f"sic_{sic_code}"
            
            print(f"Company: {row['company']}")
            print(f"SIC: {sic_code} - {sick_df.at[idx, 'sic_description']}")
            print(f"Ticker: {sick_df.at[idx, 'ticker']}")
            print(f"Is Biotech: {sick_df.at[idx, 'is_biotech']}")
            print("---")
            
    except Exception as e:
        print(f"Error for {row['company']}: {str(e)}")
    
    time.sleep(0.1)  # Respect SEC rate limits

# Filter for biotech companies
btech_delisted_df = sick_df[sick_df['is_biotech']].copy()

# Display results
print("\nPotential Biotech Companies Found:")
print(btech_delisted_df[['date', 'company', 'ticker', 'sic', 'sic_description', 'biotech_identification_method']])

# Save results with detailed information
btech_delisted_df.to_csv('biotech_delistings_detailed.csv', index=False)

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total companies processed: {len(sick_df)}")
print(f"Companies identified as biotech: {len(btech_delisted_df)}")
if len(sick_df) > 0:
    print(f"Percentage biotech: {(len(btech_delisted_df) / len(sick_df)) * 100:.1f}%")

# Print identification method breakdown
print("\nIdentification Method Breakdown:")
print(btech_delisted_df['biotech_identification_method'].value_counts())

# Print SIC code distribution for identified biotech companies
print("\nSIC Code Distribution for Identified Biotech Companies:")
sic_distribution = btech_delisted_df['sic'].value_counts()
for sic in sic_distribution.index:
    description = biotech_sic_dict.get(sic, 'Unknown')
    print(f"SIC {sic}: {description} - {sic_distribution[sic]} companies")

Processing Finch Therapeutics Group, Inc. (1/574)
Company: Finch Therapeutics Group, Inc.
SIC: 2836 - Biological Products, (No Diagnostic Substances)
Ticker: FNCH
Is Biotech: True
---
Processing TELLURIAN INC. /DE/ (2/574)
Company: TELLURIAN INC. /DE/
SIC: 1311 - Crude Petroleum & Natural Gas
Ticker: TELZ
Is Biotech: False
---
Processing Guardion Health Sciences, Inc. (3/574)
Company: Guardion Health Sciences, Inc.
SIC: 2834 - Pharmaceutical Preparations
Ticker: GHSI
Is Biotech: True
---
Processing Investment Managers Series Trust II (4/574)
Error for Investment Managers Series Trust II: list index out of range
Processing ORANGE (5/574)
Company: ORANGE
SIC: 4813 - Telephone Communications (No Radiotelephone)
Ticker: FNCTF
Is Biotech: False
---
Processing Renalytix plc (6/574)
Company: Renalytix plc
SIC: 8071 - Services-Medical Laboratories
Ticker: RNLXY
Is Biotech: True
---
Processing Generations Bancorp NY, Inc. (7/574)
Company: Generations Bancorp NY, Inc.
SIC: 6035 - Savings Institu

In [30]:
print(len(btech_delisted_df))
display(btech_delisted_df.head())  

58


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
10,2024-09-13,Calliditas Therapeutics AB,0001795579,25,,,,True,"name_match:therapeutics,therapeutic"


# remember you are dropping the ones you couldny get a ticker symbol for right now

In [31]:
# drop the rows with no ticker. then sort by date and reset index
btech_delisted_df = btech_delisted_df[btech_delisted_df['ticker'] != '']

print(len(btech_delisted_df))  
display(btech_delisted_df.head())

42


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
23,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834


In [35]:
# --
# the empty sicks
# --
btech_delisted_df[btech_delisted_df['ticker'] == '']


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method


In [32]:
# view again
display(btech_delisted_df.head())

,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
23,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834


In [33]:
# save to csv 
btech_delisted_df.to_csv('btech_delisted_df-delistings_since_2020.csv', index=False)

In [36]:
print(len(btech_delisted_df))

42


In [43]:
print(btech_delisted_df)

          date                                       company         cik  \
0   2024-10-31                Finch Therapeutics Group, Inc.  0001733257   
2   2024-10-11                Guardion Health Sciences, Inc.  0001642375   
5   2024-10-07                                 Renalytix plc  0001811115   
9   2024-09-20                              Telesis Bio Inc.  0001850079   
23  2024-07-25                                  MorphoSys AG  0001340243   
37  2024-05-31               American Oncology Network, Inc.  0001839998   
43  2024-05-06                   Clever Leaves Holdings Inc.  0001819615   
44  2024-04-29                               Vaxxinity, Inc.  0001851657   
53  2024-04-04                   Ampio Pharmaceuticals, Inc.  0001411906   
65  2024-03-11                                       LianBio  0001831283   
98  2023-12-26                  Amneal Pharmaceuticals, Inc.  0001723128   
117 2023-11-06                       PyroGenesis Canada Inc.  0001743344   
129 2023-09-

from the delisted,
which of were due to merger/acquision ONLY

## DONE

get board members from delisted
check for matches with current NEV stocks

?

look back 1 month
look back up to and including current and previous quarter chart data
other technical and chart data
financials
news

?

# delisting reasons

In [48]:
def add_sec_delisting_reasons(df):
    """
    Add SEC-based delisting reasons to existing biotech dataframe
    """
    # Add new columns for SEC information
    df['delisting_reason'] = ''
    df['sec_filings'] = ''
    
    # Set up SEC API headers
    headers = {
        'User-Agent': 'sanspeursystems matt@sanspeursystems.com',  
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'data.sec.gov'
    }
    
    for idx, row in df.iterrows():
        print(f"Processing {row['ticker']} ({idx + 1}/{len(df)})")
        
        cik = str(row['cik']).zfill(10)  # Pad CIK with leading zeros
        delisting_date = pd.to_datetime(row['date'])
        start_date = delisting_date - timedelta(days=90)  # Look back 90 days
        
        try:
            # Get SEC filings
            url = f'https://data.sec.gov/submissions/CIK{cik}.json'
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                recent_filings = data.get('filings', {}).get('recent', {})
                
                if recent_filings:
                    forms = recent_filings.get('form', [])
                    dates = recent_filings.get('filingDate', [])
                    
                    relevant_filings = []
                    primary_reason = 'Unknown'
                    
                    # Check filings around delisting date
                    for form, file_date in zip(forms, dates):
                        file_date = pd.to_datetime(file_date)
                        
                        if start_date <= file_date <= delisting_date:
                            if form == '25':
                                primary_reason = 'Voluntary Delisting'
                                relevant_filings.append(f'Form 25 ({file_date.date()})')
                                
                            elif form in ['DEFM14A', 'PREM14A']:
                                primary_reason = 'Merger/Acquisition'
                                relevant_filings.append(f'Merger Proxy {form} ({file_date.date()})')
                                
                            elif form == '15':
                                if primary_reason == 'Unknown':
                                    primary_reason = 'Deregistration'
                                relevant_filings.append(f'Form 15 ({file_date.date()})')
                                
                            elif any(x in form for x in ['15-12B', '15-12G']):
                                primary_reason = 'Bankruptcy/Liquidation'
                                relevant_filings.append(f'{form} ({file_date.date()})')
                            
                            elif form == '8-K':
                                relevant_filings.append(f'8-K ({file_date.date()})')
                    
                    # Update dataframe
                    df.at[idx, 'delisting_reason'] = primary_reason
                    df.at[idx, 'sec_filings'] = '; '.join(relevant_filings)
            
        except Exception as e:
            print(f"Error processing {row['ticker']}: {str(e)}")
        
        time.sleep(0.1)  # Respect SEC rate limits
    
    return df

In [49]:
print(len(btech_delisted_df))

42


In [50]:
# -- 
# add SEC delisting reasons
# --
btdl_df = add_sec_delisting_reasons(btech_delisted_df)

print(len(btdl_df))

btdl_df

Processing FNCH (1/42)
Processing GHSI (3/42)
Processing RNLXY (6/42)
Processing TBIO (10/42)
Processing MPSYF (24/42)
Processing AONC (38/42)
Processing CLVR (44/42)
Processing VAXX (45/42)
Processing AMPE (54/42)
Processing LIANY (66/42)
Processing AMRX (99/42)
Processing PYRGF (118/42)
Processing NKGN (130/42)
Processing COO (132/42)
Processing AEON (150/42)
Processing BLUAF (180/42)
Processing TMDIF (183/42)
Processing ZURA (192/42)
Processing ZYME (238/42)
Processing CLDI (245/42)
Processing FOXO (261/42)
Processing FOXO (262/42)
Processing VANI (270/42)
Processing KLDO (309/42)
Processing SCPX (328/42)
Processing ANVS (354/42)
Processing ERNA (363/42)
Processing DNA (382/42)
Processing SRZN (390/42)
Processing ME (403/42)
Processing NSPR (404/42)
Processing ACB (408/42)
Processing NSPR (409/42)
Processing BIOX (413/42)
Processing OCX (426/42)
Processing CRMD (428/42)
Processing VTAK (430/42)
Processing CLOV (435/42)
Processing AMGN (441/42)
Processing CGC (459/42)
Processing AZN 

,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method,delisting_reason,sec_filings
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836",Voluntary Delisting,Form 25 (2024-10-31); 8-K (2024-10-21); 8-K (2...
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834,Voluntary Delisting,Form 25 (2024-10-11); 8-K (2024-10-03); 8-K (2...
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071,Voluntary Delisting,Form 25 (2024-10-07); 8-K (2024-10-01); 8-K (2...
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826,Voluntary Delisting,Form 25 (2024-09-20); 8-K (2024-09-10); 8-K (2...
23,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834,Voluntary Delisting,Form 25 (2024-07-25)
37,2024-05-31,"American Oncology Network, Inc.",0001839998,25,AONC,8000,Services-Health Services,True,name_match:oncology,Voluntary Delisting,Form 25 (2024-05-31); 8-K (2024-05-21); 8-K (2...
43,2024-05-06,Clever Leaves Holdings Inc.,0001819615,25,CLVR,2834,Pharmaceutical Preparations,True,sic_2834,Voluntary Delisting,Form 25 (2024-05-06); 8-K (2024-04-26); 8-K (2...
44,2024-04-29,"Vaxxinity, Inc.",0001851657,25,VAXX,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834,Voluntary Delisting,Form 25 (2024-04-29); 8-K (2024-04-19); 8-K (2...
53,2024-04-04,"Ampio Pharmaceuticals, Inc.",0001411906,25,AMPE,2834,Pharmaceutical Preparations,True,"name_match:pharma,pharmaceutical;sic_2834",Voluntary Delisting,Form 25 (2024-04-04); 8-K (2024-03-25); 8-K (2...
65,2024-03-11,LianBio,0001831283,25,LIANY,2834,Pharmaceutical Preparations,True,name_match:bio;sic_2834,Voluntary Delisting,Form 25 (2024-03-11); 8-K (2024-02-13); 8-K (2...


In [51]:
btdl_m_and_a_df = btdl_df[btdl_df['delisting_reason'] == 'Merger/Acquisition']
print(len(btdl_m_and_a_df))
display(btdl_m_and_a_df.head())

5


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method,delisting_reason,sec_filings
149,2023-07-21,Priveterra Acquisition Corp.,0001837607,25,AEON,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834,Merger/Acquisition,Form 25 (2023-07-21); 8-K (2023-07-17); 8-K (2...
269,2022-09-08,"Vivani Medical, Inc.",0001266806,25,VANI,3845,Electromedical & Electrotherapeutic Apparatus,True,name_match:medical;sic_3845,Merger/Acquisition,Form 25 (2022-09-08); 8-K (2022-09-02); 8-K (2...
389,2021-08-11,Consonance-HFW Acquisition Corp.,0001824893,25,SRZN,2836,"Biological Products, (No Diagnostic Substances)",True,sic_2836;sic_2836,Merger/Acquisition,Form 25 (2021-08-11); 8-K (2021-08-10); 8-K (2...
402,2021-06-16,VG Acquisition Corp.,0001804591,25,ME,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834,Merger/Acquisition,Form 25 (2021-06-16); 8-K (2021-06-11); 8-K (2...
434,2021-01-07,Social Capital Hedosophia Holdings Corp. III,0001801170,25,CLOV,6324,Hospital & Medical Service Plans,True,sic_6324;sic_6324,Merger/Acquisition,Form 25 (2021-01-07); 8-K (2021-01-06); 8-K (2...


remember these are just since 2020
can scale further back

# retrieve historical data

In [55]:
import yfinance as yf
from datetime import datetime, timedelta

def get_last_available_data(ticker):
    """
    Retrieve the last available trading data for any stock, including delisted ones.
    
    Parameters:
    ticker (str): Stock ticker symbol
    
    Returns:
    tuple: (pandas.DataFrame with last available data, last trading date string)
    """
    try:
        # Create ticker object
        ticker_obj = yf.Ticker(ticker)
        
        # Get max available historical data (using a reasonable period)
        hist = ticker_obj.history(period="1mo")  # Try last month first
        
        if hist.empty:
            # If no data in last month, try getting max historical data
            hist = ticker_obj.history(period="max")
            
        if hist.empty:
            print(f"No historical data found for ticker {ticker}")
            return None, None
            
        # Get the last available row
        last_data = hist.tail(1)
        
        # Get the actual last trading date
        last_date = last_data.index[0].strftime('%Y-%m-%d')
        
        return last_data, last_date
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None, None


In [57]:
# Example usage
ticker_symbol = 'VANI'

# Get the last available data
last_data, last_date = get_last_available_data(ticker_symbol)

if last_data is not None:
    print(f"\nLast available trading data for {ticker_symbol} (Date: {last_date}):")
    print(last_data)
    
    # Print specific values in a more readable format
    row = last_data.iloc[0]
    print(f"\nDetailed information:")
    print(f"Date: {last_date}")
    print(f"Open: ${row['Open']:.2f}")
    print(f"High: ${row['High']:.2f}")
    print(f"Low: ${row['Low']:.2f}")
    print(f"Close: ${row['Close']:.2f}")
    print(f"Volume: {row['Volume']:,}")
    
    # Calculate daily return
    print(f"Daily Return: {((row['Close'] - row['Open']) / row['Open'] * 100):.2f}%")

# Optional: Get full info about the stock
def print_extra_info(ticker):
    """Print additional information about the stock if available"""
    try:
        ticker_obj = yf.Ticker(ticker)
        info = ticker_obj.info
        
        print("\nAdditional Stock Information:")
        relevant_fields = [
            'longName', 'industry', 'sector', 
            'previousClose', 'regularMarketPrice',
            'marketCap', 'currency'
        ]
        
        for field in relevant_fields:
            if field in info:
                print(f"{field}: {info[field]}")
                
    except Exception as e:
        print(f"Could not fetch additional information: {e}")

# Uncomment to get extra info
# print_extra_info(ticker_symbol)


Last available trading data for AEON (Date: 2024-11-04):
                             Open    High     Low  Close  Volume  Dividends  \
Date                                                                          
2024-11-04 00:00:00-05:00  0.8796  0.9066  0.8709  0.879   16968        0.0   

                           Stock Splits  
Date                                     
2024-11-04 00:00:00-05:00           0.0  

Detailed information:
Date: 2024-11-04
Open: $0.88
High: $0.91
Low: $0.87
Close: $0.88
Volume: 16,968.0
Daily Return: -0.07%


In [42]:
####----> 0
########################################
# --- Input variables --- #
# working
ticker_hack = 'PLTR'

########################################
####----> 1
# get stock data from yahoo finance
def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")
        info = stock.info
        return ticker, hist, info
    except:
        return ticker, None, None
    

####----> 2
#### 
# return stock data for the stock ticker
####
ticker, hist, info = get_stock_data(ticker_hack)
print(ticker)
# print(hist)
print(info)

# # print info but beutify? it
# for key, value in info.items():
#     # print info in a pretty JSON format
#     print(json.dumps(info, indent=4))   


####----> 3
def get_latest_stock_data(ticker):
    """
    Fetches the most recent historical data and info for a given ticker.
    
    Parameters:
        ticker (str): The stock ticker symbol.
        
    Returns:
        tuple: (ticker, latest_hist, info)
            - ticker (str): The stock ticker.
            - latest_hist (dict): The most recent historical data point.
            - info (dict): Informational data about the stock.
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        
        # Fetch the most recent historical data (last 1 day)
        hist = ticker_obj.history(period='1d', interval='5m')  # 1-minute interval for the latest day
        
        if hist.empty:
            logging.warning(f"No historical data found for ticker {ticker}.")
            latest_hist = {}
        else:
            # Get the last available row
            latest_row = hist.tail(1)
            latest_hist = latest_row.to_dict(orient='records')[0]
        
        # Fetch the info dictionary
        info = ticker_obj.info
        
        return ticker, latest_hist, info

    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {e}")
        return ticker, {}, {}


# ####----> 4
# #############################################
# def save_to_json(ticker, data):
#     # Define the path
#     directory = f'/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/bio_targets/{ticker}'
#     path = f'{directory}/yahoo_finance_new_data_{ticker}.json'
    
#     # Create the directory if it doesn't exist
#     Path(directory).mkdir(parents=True, exist_ok=True)
    
#     # Save data as JSON
#     with open(path, 'w') as f:
#         json.dump(data, f, indent=4)

# # Prepare data to save
# data_to_save = {
#     'ticker': ticker,
#     'info': info
# }

# # Call the function to save data
# save_to_json(ticker, data_to_save)
# ########################################

# ####----> 5
# if ticker_hack == ticker:
#     print(True)
# else:
#     print(False)

PLTR
{'address1': '1200 17th Street', 'address2': 'Floor 15', 'city': 'Denver', 'state': 'CO', 'zip': '80202', 'country': 'United States', 'phone': '720 358 3679', 'website': 'https://www.palantir.com', 'industry': 'Software - Infrastructure', 'industryKey': 'software-infrastructure', 'industryDisp': 'Software - Infrastructure', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "Palantir Technologies Inc. builds and deploys software platforms for the intelligence community to assist in counterterrorism investigations and operations in the United States, the United Kingdom, and internationally. The company provides Palantir Gotham, a software platform which enables users to identify patterns hidden deep within datasets, ranging from signals intelligence sources to reports from confidential informants, as well as facilitates the handoff between analysts and operational users, helping operators plan and execute real-world responses to th

| END |
|-|
| this all works. |